# 🤖 Fine-tuning Babišova stylu - Google Colab

Tento notebook implementuje kompletní pipeline pro fine-tuning jazykového modelu na napodobení charakteristického stylu komunikace Andreje Babiše.

## 🎯 Cíl
Vytvořit model, který dokáže generovat satirické odpovědi ve stylu "babíšovštiny" - charakteristického jazykového stylu s mluvenou češtinou, slovensko-českými odchylkami a specifickými rétorickými prvky.

## 🚀 Výhody Google Colab
- ✅ **Zdarma GPU** (Tesla T4/P100)
- ✅ **Žádná instalace** - vše v prohlížeči
- ✅ **Automatické ukládání** na Google Drive
- ✅ **Sdílení** výsledků přes Hugging Face Hub
- ✅ **Monitoring** přes Weights & Biases

## 📦 Instalace a setup

In [ ]:
#!/usr/bin/env python3

# Instalace potřebných knihoven
!pip install -q transformers datasets peft accelerate bitsandbytes wandb tiktoken
!pip install -q huggingface_hub gradio streamlit

# Restart runtime po instalaci
import os
os.kill(os.getpid(), 9)

In [ ]:
#!/usr/bin/env python3

# Import knihoven
import torch
import json
import logging
from typing import Optional, Dict, Any
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from datasets import Dataset, DatasetDict
import wandb
from huggingface_hub import HfApi, login
from google.colab import drive
from tqdm import tqdm

# Nastavení logování
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"PyTorch verze: {torch.__version__}")
print(f"CUDA dostupné: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU paměť: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB") 

In [ ]:
#!/usr/bin/env python3

# Připojení Google Drive
drive.mount('/content/drive')

# Vytvoření adresářů
!mkdir -p /content/babis_finetune
!mkdir -p /content/drive/MyDrive/babis_finetune

print("Google Drive připojen a adresáře vytvořeny!") 

## ⚙️ Konfigurace

In [ ]:
#!/usr/bin/env python3

"""
Konfigurace pro fine-tuning Babišova stylu
"""
from dataclasses import dataclass
from typing import List

@dataclass
class ColabConfig:
    """Konfigurace pro Google Colab fine-tuning"""
    
    # Model settings
    base_model: str = "microsoft/DialoGPT-medium"  # Malý model pro Colab
    model_name: str = "babis-dialogpt-colab"
    
    # Training settings (optimalizováno pro Colab GPU)
    learning_rate: float = 2e-4
    num_train_epochs: int = 2
    per_device_train_batch_size: int = 2
    per_device_eval_batch_size: int = 2
    gradient_accumulation_steps: int = 8
    max_grad_norm: float = 0.3
    warmup_steps: int = 50
    logging_steps: int = 10
    save_steps: int = 200
    eval_steps: int = 200
    
    # LoRA settings
    lora_r: int = 8
    lora_alpha: int = 16
    lora_dropout: float = 0.1
    target_modules: List[str] = None
    
    # Dataset settings
    max_seq_length: int = 512
    train_split: float = 0.9
    eval_split: float = 0.1
    
    # Output settings
    output_dir: str = "/content/babis_finetune"
    logging_dir: str = "/content/babis_finetune/logs"
    
    # Hardware settings (optimalizováno pro Colab)
    fp16: bool = True
    bf16: bool = False
    use_8bit: bool = False
    use_4bit: bool = True
    
    # Evaluation settings
    evaluation_strategy: str = "steps"
    save_strategy: str = "steps"
    load_best_model_at_end: bool = True
    metric_for_best_model: str = "eval_loss"
    greater_is_better: bool = False
    
    def __post_init__(self):
        if self.target_modules is None:
            # Pro DialoGPT
            self.target_modules = ["c_attn", "c_proj", "wte", "wpe"]
        
        # Vytvoření adresářů
        import os
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.logging_dir, exist_ok=True)

# Vytvoření konfigurace
config = ColabConfig()
print("Konfigurace vytvořena:")
print(f"Base model: {config.base_model}")
print(f"Output dir: {config.output_dir}")
print(f"LoRA r: {config.lora_r}, alpha: {config.lora_alpha}") 

: 

## 📊 Příprava datasetu

In [ ]:
"""
Modul pro načítání a zpracování datasetu
"""
import json
from pathlib import Path
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

def create_babis_dataset():
    """Načte skutečný dataset s Babišovými výroky z JSONL souborů"""
    
    # Cesta k souborům s daty
    data_dir = Path("final")
    jsonl_files = list(data_dir.glob("batch_*_babis_output_qa.jsonl"))
    
    if not jsonl_files:
        raise FileNotFoundError(f"Nenalezeny žádné JSONL soubory v adresáři {data_dir}")
    
    conversations = []
    
    # Načtení všech souborů
    for file_path in jsonl_files:
        print(f"Načítám {file_path.name}...")
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                try:
                    data = json.loads(line.strip())
                    question = data.get('question', '')
                    answer = data.get('answer', '')
                    
                    # Vytvoření konverzace ve formátu pro fine-tuning
                    conversation = {
                        "prompt": question,
                        "completion": answer,
                        "full_conversation": f"Uživatel: {question}\nAndrej Babiš: {answer}"
                    }
                    conversations.append(conversation)
                    
                except json.JSONDecodeError as e:
                    print(f"Chyba při parsování JSON na řádku {line_num} v souboru {file_path.name}: {e}")
                    continue
    
    print(f"Celkem načteno {len(conversations)} konverzací z {len(jsonl_files)} souborů")
    
    if len(conversations) == 0:
        raise ValueError("Nebyla načtena žádná konverzace z JSONL souborů")
    
    # Rozdělení na train/validation
    train_data, eval_data = train_test_split(conversations, train_size=0.9, random_state=42)
    
    # Vytvoření Dataset objektů
    train_dataset = Dataset.from_list(train_data)
    eval_dataset = Dataset.from_list(eval_data)
    
    # Vytvoření DatasetDict
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': eval_dataset
    })
    
    return dataset_dict

# Vytvoření datasetu
dataset = create_babis_dataset()
print(f"Dataset vytvořen:")
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"\nPříklad dat:")
print(dataset['train'][0])

## 🤖 Načtení modelu a tokenizeru

In [ ]:
# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained(config.base_model)

# Přidání padding tokenu pokud chybí
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer načten: {config.base_model}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Pad token: {tokenizer.pad_token}")

In [ ]:
# Načtení modelu s kvantizací (optimalizováno pro Colab GPU)
model_kwargs = {
    "torch_dtype": torch.float16 if config.fp16 else torch.float32,
    "device_map": "auto" if torch.cuda.is_available() else None
}

# Kvantizace pro úsporu paměti
if config.use_4bit:
    model_kwargs["load_in_4bit"] = True
elif config.use_8bit:
    model_kwargs["load_in_8bit"] = True

model = AutoModelForCausalLM.from_pretrained(
    config.base_model,
    **model_kwargs
)

# Příprava modelu pro kvantizované trénování
if config.use_4bit or config.use_8bit:
    model = prepare_model_for_kbit_training(model)

print(f"Model načten: {config.base_model}")
print(f"Model device: {next(model.parameters()).device}")
print(f"Model dtype: {next(model.parameters()).dtype}")

## 🎯 Nastavení LoRA

In [ ]:
# Nastavení LoRA konfigurace
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=config.target_modules,
    lora_dropout=config.lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Aplikace LoRA na model
model = get_peft_model(model, lora_config)

# Výpis trénovatelných parametrů
model.print_trainable_parameters()

print("\nLoRA konfigurace nastavena!")
print(f"LoRA r: {config.lora_r}")
print(f"LoRA alpha: {config.lora_alpha}")
print(f"Target modules: {config.target_modules}") 

## 🔤 Tokenizace datasetu

In [ ]:
def tokenize_function(examples):
    """Tokenizuje texty v datasetu"""
    # Používáme full_conversation pro trénování
    texts = examples['full_conversation']
    
    # Tokenizace
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=config.max_seq_length,
        return_tensors="pt"
    )
    
    # Nastavení labels na input_ids pro causal LM
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

# Tokenizace datasetu
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print("Dataset tokenizován!")
print(f"Train samples: {len(tokenized_dataset['train'])}")
print(f"Validation samples: {len(tokenized_dataset['validation'])}")
print(f"\nPříklad tokenizovaných dat:")
print(f"Input shape: {tokenized_dataset['train'][0]['input_ids'].shape}")
print(f"Labels shape: {tokenized_dataset['train'][0]['labels'].shape}") 

## 🏋️ Fine-tuning

In [ ]:
# Nastavení Weights & Biases (volitelné)
try:
    wandb.init(
        project="babis-finetune-colab",
        name=config.model_name,
        config=vars(config)
    )
    print("Weights & Biases inicializováno")
except Exception as e:
    print(f"WandB inicializace selhala: {e}")
    wandb = None

In [ ]:
# Nastavení training arguments
training_args = TrainingArguments(
    output_dir=config.output_dir,
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    max_grad_norm=config.max_grad_norm,
    warmup_steps=config.warmup_steps,
    logging_steps=config.logging_steps,
    save_steps=config.save_steps,
    eval_steps=config.eval_steps,
    evaluation_strategy=config.evaluation_strategy,
    save_strategy=config.save_strategy,
    load_best_model_at_end=config.load_best_model_at_end,
    metric_for_best_model=config.metric_for_best_model,
    greater_is_better=config.greater_is_better,
    fp16=config.fp16,
    bf16=config.bf16,
    logging_dir=config.logging_dir,
    report_to="wandb" if wandb else None,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    save_total_limit=3,
    prediction_loss_only=True
)

print("Training arguments nastaveny:")
print(f"Learning rate: {config.learning_rate}")
print(f"Epochs: {config.num_train_epochs}")
print(f"Batch size: {config.per_device_train_batch_size}")
print(f"Gradient accumulation: {config.gradient_accumulation_steps}")

In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Callbacks
callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    callbacks=callbacks
)

print("Trainer nastaven!")
print(f"Train samples: {len(tokenized_dataset['train'])}")
print(f"Validation samples: {len(tokenized_dataset['validation'])}")

In [ ]:
# Spuštění trénování
print("🚀 Začínám fine-tuning...")
print(f"Model: {config.base_model}")
print(f"Dataset: {len(tokenized_dataset['train'])} train, {len(tokenized_dataset['validation'])} validation")
print(f"Epochs: {config.num_train_epochs}")
print(f"LoRA r: {config.lora_r}, alpha: {config.lora_alpha}")
print("-" * 50)

# Trénování
train_result = trainer.train()

print("\n✅ Fine-tuning dokončen!")
print(f"Training loss: {train_result.metrics.get('train_loss', 'N/A')}")
print(f"Training time: {train_result.metrics.get('train_runtime', 'N/A')}s")

In [ ]:
# Evaluace modelu
print("📊 Spouštím evaluaci...")

eval_results = trainer.evaluate()

print("\n📈 Výsledky evaluace:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Uložení metrik
trainer.log_metrics("eval", eval_results)
trainer.save_metrics("eval", eval_results)

## 💾 Uložení modelu

In [ ]:
# Uložení modelu
print("💾 Ukládám model...")

# Uložení na Colab
trainer.save_model()
tokenizer.save_pretrained(config.output_dir)

# Uložení na Google Drive
drive_path = f"/content/drive/MyDrive/babis_finetune/{config.model_name}"
!cp -r {config.output_dir} {drive_path}

print(f"✅ Model uložen:")
print(f"Colab: {config.output_dir}")
print(f"Google Drive: {drive_path}")

# Výpis velikosti modelu
!du -sh {config.output_dir} 

## 🚀 Push na Hugging Face Hub (volitelné)

In [ ]:
# Nastavení Hugging Face tokenu
from getpass import getpass

print("Pro push na Hugging Face Hub potřebujete token.")
print("Získejte ho na: https://huggingface.co/settings/tokens")
print("Pokud nechcete pushovat, stiskněte Enter.")

hf_token = getpass("HF Token (volitelné): ")

if hf_token.strip():
    login(hf_token)
    print("✅ Přihlášeno k Hugging Face Hub")
    
    # Push na Hub
    print("🚀 Pushuji model na Hub...")
    trainer.push_to_hub(f"babis-{config.model_name}")
    tokenizer.push_to_hub(f"babis-{config.model_name}")
    
    print(f"✅ Model pushnut na Hub: babis-{config.model_name}")
    print(f"URL: https://huggingface.co/babis-{config.model_name}")
else:
    print("Model nebyl pushnut na Hub.") 

## 🧪 Testování modelu

In [ ]:
# Funkce pro generování odpovědí
def generate_babis_response(prompt, max_length=100, temperature=0.7):
    """Vygeneruje odpověď ve stylu Babiše"""
    
    # Tokenizace promptu
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=config.max_seq_length)
    
    # Generování
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Dekódování
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Odstranění původního promptu
    response = response.replace(prompt, "").strip()
    
    return response

# Testovací prompty
test_prompts = [
    "Uživatel: Jaký je váš názor na inflaci?\nAndrej Babiš: ",
    "Uživatel: Co si myslíte o Bruselu?\nAndrej Babiš: ",
    "Uživatel: Jak hodnotíte opozici?\nAndrej Babiš: ",
    "Uživatel: Jaké máte plány?\nAndrej Babiš: "
]

print("🧪 Testování fine-tuned modelu:")
print("=" * 50)

for prompt in test_prompts:
    response = generate_babis_response(prompt)
    print(f"\nPrompt: {prompt.strip()}")
    print(f"Odpověď: {response}")
    print("-" * 30) 

## 🌐 Gradio Interface (volitelné)

In [ ]:
# Vytvoření Gradio interface
import gradio as gr

def babis_chat(message, history):
    """Chat funkce pro Gradio"""
    prompt = f"Uživatel: {message}\nAndrej Babiš: "
    response = generate_babis_response(prompt, max_length=150, temperature=0.8)
    return response

# Vytvoření interface
iface = gr.ChatInterface(
    fn=babis_chat,
    title="🤖 Babiš Chat Bot",
    description="Fine-tuned model ve stylu Andreje Babiše",
    examples=[
        ["Jaký je váš názor na inflaci?"],
        ["Co si myslíte o Bruselu?"],
        ["Jak hodnotíte opozici?"],
        ["Jaké máte plány do budoucna?"],
        ["Jak trávíte čas s rodinou?"],
    ]
)

# Spuštění interface
iface.launch(share=True, debug=True)

print("🌐 Gradio interface spuštěn!")
print("Sdílejte odkaz s ostatními pro testování.") 

## 📊 Shrnutí výsledků

In [ ]:
# Shrnutí výsledků
print("🎉 FINE-TUNING DOKONČEN!")
print("=" * 50)
print(f"Model: {config.base_model}")
print(f"Fine-tuned model: {config.model_name}")
print(f"Training loss: {train_result.metrics.get('train_loss', 'N/A'):.4f}")
print(f"Evaluation loss: {eval_results.get('eval_loss', 'N/A'):.4f}")
print(f"Training time: {train_result.metrics.get('train_runtime', 'N/A'):.1f}s")
print(f"LoRA parameters: r={config.lora_r}, alpha={config.lora_alpha}")
print(f"Dataset size: {len(tokenized_dataset['train'])} train, {len(tokenized_dataset['validation'])} validation")
print("\n📁 Uložené soubory:")
print(f"Colab: {config.output_dir}")
print(f"Google Drive: /content/drive/MyDrive/babis_finetune/{config.model_name}")

if hf_token.strip():
    print(f"Hugging Face Hub: babis-{config.model_name}")

print("\n🚀 Další kroky:")
print("1. Stáhněte model z Google Drive")
print("2. Použijte ho ve vlastní aplikaci")
print("3. Sdílejte výsledky s ostatními")
print("4. Experimentujte s různými prompty")

print("\n✅ Hotovo! Model je připraven k použití.")